# Chat Completion api

In [ ]:
! pip install openapi

## 환경설정
1. openai사이트에서 API KEY발급
2. google colab - secret 탭에 'OPENAI_API_KEY' 등록


In [9]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [10]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고 많은 도움을 주는 챗봇이야"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 내 이름은 차은우야"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 힘든 작업으로 끼니를 거르는 노동자들의 고충


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길~
- 교정 제목: "주말 폭우 예보, 안전에 유의하세요"


## 영문텍스트에 anki 단어장 만들기

# Task
Extract vocabulary from the provided English text, including word meanings, and format it as a CSV file suitable for importing into Anki.

## Define a function to extract vocabulary

### Subtask:
Create a Python function that takes English text as input and outputs a list of vocabulary words and their meanings.


**Reasoning**:
Define a Python function that takes English text as input and uses the OpenAI API to extract vocabulary and their meanings.



In [7]:
def extract_vocabulary(english_text, model='gpt-4o-mini', temperature=0.5, top_p=1):
    """
    Extracts vocabulary from English text and provides their meanings using the OpenAI API.

    Args:
        english_text: A string containing the English text.
        model: The OpenAI model to use.
        temperature: The temperature for the API call.
        top_p: The top_p for the API call.

    Returns:
        A string containing the extracted vocabulary and their meanings formatted for Anki import.
    """
    system_message = """
    You are a vocabulary extractor. Your task is to read the provided English text and identify important vocabulary words.
    For each word, provide a concise definition. Format the output as a list where each item is the vocabulary word followed by a colon and then its meaning, separated by a newline for each word. This format should be suitable for direct import into Anki as a basic card (front: word, back: meaning).

    ### Output Format ###
    word1: meaning1
    word2: meaning2
    word3: meaning3
    ...

    ### Example ###
    Input Text: "The quick brown fox jumps over the lazy dog."
    Output:
    quick: moving rapidly; happening in a short time
    brown: of the color of earth or coffee
    fox: a carnivorous mammal of the dog family with a pointed muzzle and bushy tail
    jumps: push oneself off a surface and into the air using one's feet and legs
    lazy: unwilling to work or use energy
    dog: a domesticated carnivorous mammal that typically has a long snout, an acute sense of smell, and a barking, howling, or whining voice.
    """

    user_message = f"""
    Extract vocabulary and their meanings from the following text:

    {english_text}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

# Example usage:
text = "Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence. AI is an interdisciplinary science with multiple approaches, but advancements in machine learning and deep learning are creating a paradigm shift in virtually every sector of the tech industry."
vocabulary_list = extract_vocabulary(text)
print(vocabulary_list)

artificial intelligence: the simulation of human intelligence processes by machines, especially computer systems

AI: abbreviation for artificial intelligence

wide-ranging: covering a large area or scope; extensive

branch: a division or subdivision of a larger organization or field of study

computer science: the study of computers and computational systems

smart machines: devices that are capable of performing tasks intelligently, often using AI technology

capable: having the ability, fitness, or quality necessary to do or achieve a specified thing

performing: carrying out, executing, or accomplishing a task or function

tasks: specific pieces of work or duties that need to be done

typically: in a way that is characteristic of a particular person or thing

require: to need something for a particular purpose

human intelligence: the mental capacity for reasoning, problem-solving, and understanding complex ideas

interdisciplinary: involving two or more academic, scientific, or ar

**Reasoning**:
The previous code failed because the `client` object was not defined within the function's scope. I need to make sure the `client` object, which is an instance of the OpenAI class, is accessible inside the `extract_vocabulary` function. Since the `client` object was created in a previous cell, I will make sure it is globally available or pass it to the function. The simplest fix is to ensure the cell where the client is defined has been run. However, since I cannot assume the execution order of the cells, I will explicitly pass the client object to the function.



In [8]:
from openai import OpenAI

def extract_vocabulary(client, english_text, model='gpt-4o-mini', temperature=0.5, top_p=1):
    """
    Extracts vocabulary from English text and provides their meanings using the OpenAI API.

    Args:
        client: An instance of the OpenAI client.
        english_text: A string containing the English text.
        model: The OpenAI model to use.
        temperature: The temperature for the API call.
        top_p: The top_p for the API call.

    Returns:
        A string containing the extracted vocabulary and their meanings formatted for Anki import.
    """
    system_message = """
    You are a vocabulary extractor. Your task is to read the provided English text and identify important vocabulary words.
    For each word, provide a concise definition. Format the output as a list where each item is the vocabulary word followed by a colon and then its meaning, separated by a newline for each word. This format should be suitable for direct import into Anki as a basic card (front: word, back: meaning).

    ### Output Format ###
    word1: meaning1
    word2: meaning2
    word3: meaning3
    ...

    ### Example ###
    Input Text: "The quick brown fox jumps over the lazy dog."
    Output:
    quick: moving rapidly; happening in a short time
    brown: of the color of earth or coffee
    fox: a carnivorous mammal of the dog family with a pointed muzzle and bushy tail
    jumps: push oneself off a surface and into the air using one's feet and legs
    lazy: unwilling to work or use energy
    dog: a domesticated carnivorous mammal that typically has a long snout, an acute sense of smell, and a barking, howling, or whining voice.
    """

    user_message = f"""
    Extract vocabulary and their meanings from the following text:

    {english_text}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

# Assuming 'client' is defined in a previous cell and is an instance of OpenAI
text = "Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence. AI is an interdisciplinary science with multiple approaches, but advancements in machine learning and deep learning are creating a paradigm shift in virtually every sector of the tech industry."
vocabulary_list = extract_vocabulary(client, text)
print(vocabulary_list)

artificial intelligence: the simulation of human intelligence processes by machines, especially computer systems  
AI: abbreviation for artificial intelligence  
wide-ranging: covering a large area; extensive  
branch: a division or section of a larger organization or field of study  
computer science: the study of computers and computational systems  
smart machines: devices that can perform tasks intelligently, often through automation and learning  
capable: having the ability or qualities necessary to do something  
performing: carrying out or executing a task or function  
tasks: specific pieces of work or duties to be accomplished  
typically: in a way that is characteristic of a particular person or thing  
require: need for a particular purpose  
human intelligence: the cognitive ability of humans to learn, reason, and solve problems  
interdisciplinary: involving two or more academic, scientific, or artistic disciplines  
science: a systematically organized body of knowledge o

**Reasoning**:
The `NameError: name 'client' is not defined` error persists because the `client` object, although defined in a previous cell, is not available in the current execution environment when the `extract_vocabulary` function is called. To fix this, I need to ensure the `client` object is created in the same cell or is accessible in the current scope before calling the function. I will explicitly include the code to create the `client` object within this code block to guarantee its availability.



In [3]:
from google.colab import userdata
from openai import OpenAI

# Ensure the OpenAI client is initialized
try:
    client
except NameError:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=OPENAI_API_KEY)

def extract_vocabulary(client, english_text, model='gpt-4o-mini', temperature=0.5, top_p=1):
    """
    Extracts vocabulary from English text and provides their meanings using the OpenAI API.

    Args:
        client: An instance of the OpenAI client.
        english_text: A string containing the English text.
        model: The OpenAI model to use.
        temperature: The temperature for the API call.
        top_p: The top_p for the API call.

    Returns:
        A string containing the extracted vocabulary and their meanings formatted for Anki import.
    """
    system_message = """
    You are a vocabulary extractor. Your task is to read the provided English text and identify important vocabulary words.
    For each word, provide a concise definition. Format the output as a list where each item is the vocabulary word followed by a colon and then its meaning, separated by a newline for each word. This format should be suitable for direct import into Anki as a basic card (front: word, back: meaning).

    ### Output Format ###
    word1: meaning1
    word2: meaning2
    word3: meaning3
    ...

    ### Example ###
    Input Text: "The quick brown fox jumps over the lazy dog."
    Output:
    quick: moving rapidly; happening in a short time
    brown: of the color of earth or coffee
    fox: a carnivorous mammal of the dog family with a pointed muzzle and bushy tail
    jumps: push oneself off a surface and into the air using one's feet and legs
    lazy: unwilling to work or use energy
    dog: a domesticated carnivorous mammal that typically has a long snout, an acute sense of smell, and a barking, howling, or whining voice.
    """

    user_message = f"""
    Extract vocabulary and their meanings from the following text:

    {english_text}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

# Example usage:
text = "Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence. AI is an interdisciplinary science with multiple approaches, but advancements in machine learning and deep learning are creating a paradigm shift in virtually every sector of the tech industry."
vocabulary_list = extract_vocabulary(client, text)
print(vocabulary_list)

artificial intelligence: the simulation of human intelligence processes by machines, particularly computer systems  
AI: abbreviation for artificial intelligence  
wide-ranging: covering a large area or scope; extensive  
branch: a division or subdivision of a larger whole  
computer science: the study of computers and computational systems  
smart machines: devices that can perform tasks intelligently, often mimicking human capabilities  
capable: having the ability or qualities necessary to do something  
performing: carrying out or executing a task or function  
tasks: specific jobs or duties to be completed  
typically: in a way that is characteristic or usual  
require: to need something for a particular purpose  
human intelligence: the cognitive ability of humans to learn, reason, and solve problems  
interdisciplinary: involving two or more academic disciplines or fields of study  
multiple: having or involving several parts, elements, or members  
approaches: methods or ways o

## Process the input text

### Subtask:
Apply the function to the provided English text to get the vocabulary list.


**Reasoning**:
Define the English text and call the `extract_vocabulary` function with the client and text, storing the result.



In [4]:
english_text_to_process = """
Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data. Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural language generation.
"""

vocabulary_list_output = extract_vocabulary(client, english_text_to_process)
print(vocabulary_list_output)

natural: existing in or produced by nature; not made or caused by humankind  
language: a system of communication used by a particular community or country  
processing: the action of performing a series of operations on data to extract information or achieve a result  
linguistics: the scientific study of language and its structure  
computer: an electronic device for storing and processing data  
science: a systematically organized body of knowledge on a particular subject  
artificial: made or produced by human beings rather than occurring naturally  
intelligence: the ability to acquire and apply knowledge and skills  
interactions: the reciprocal action or influence between entities  
program: to create a sequence of instructions to be executed by a computer  
analyze: to examine in detail to understand or interpret  
challenges: difficulties that require a solution or effort to overcome  
speech recognition: the ability of a machine or program to identify and process human speech

## Format the output for anki

### Subtask:
Convert the vocabulary list into a format suitable for importing into Anki (e.g., a CSV file).


**Reasoning**:
Process the vocabulary list string to create CSV formatted content suitable for Anki import, handling potential commas within fields by enclosing them in double quotes.



In [5]:
anki_csv_lines = []
for line in vocabulary_list_output.strip().split('\n'):
    parts = line.split(':', 1)
    if len(parts) == 2:
        word = parts[0].strip()
        meaning = parts[1].strip()

        # Enclose fields with commas in double quotes
        word_formatted = f'"{word}"' if ',' in word else word
        meaning_formatted = f'"{meaning}"' if ',' in meaning else meaning

        anki_csv_lines.append(f"{word_formatted},{meaning_formatted}")

anki_csv_output = "\n".join(anki_csv_lines)

print(anki_csv_output)

natural,existing in or produced by nature; not made or caused by humankind
language,a system of communication used by a particular community or country
processing,the action of performing a series of operations on data to extract information or achieve a result
linguistics,the scientific study of language and its structure
computer,an electronic device for storing and processing data
science,a systematically organized body of knowledge on a particular subject
artificial,made or produced by human beings rather than occurring naturally
intelligence,the ability to acquire and apply knowledge and skills
interactions,the reciprocal action or influence between entities
program,to create a sequence of instructions to be executed by a computer
analyze,to examine in detail to understand or interpret
challenges,difficulties that require a solution or effort to overcome
speech recognition,the ability of a machine or program to identify and process human speech
understanding,the ability to compreh

## Summary:

### Data Analysis Key Findings

*   A Python function `extract_vocabulary` was successfully created using the OpenAI API to extract vocabulary and their meanings from provided English text.
*   The function was successfully applied to the input text, yielding a list of vocabulary words and their definitions in the format "word: meaning".
*   The extracted vocabulary list was successfully converted into a CSV format suitable for Anki import, with words and meanings separated by commas and fields containing commas enclosed in double quotes.

### Insights or Next Steps

*   The generated CSV output can be directly saved as a `.csv` file and imported into Anki to create flashcards for vocabulary learning.
*   Consider adding error handling for cases where the API might return an unexpected format or fail to extract vocabulary.


## 연애코치 ReAct

In [11]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [13]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 기억에 남을 선물을 해주고싶은데 뭐가 좋을까요'))

1. 상황분석:
   - 여러분은 여자친구와의 100일을 맞이하여 특별한 선물을 고민하고 있습니다. 이는 중요한 의미를 지닌 기념일이므로 그녀가 감동할 수 있는 요소가 포함된 선물이 필요합니다. 무엇보다 선물은 그녀의 취향과 여러분의 응원의 메시지가 잘 전달될 수 있도록 신중하게 선택해야 합니다.

2. 행동계획:
   - 그녀의 취향과 소중한 기억을 고려하여 개인화된 선물을 선택합니다. 몇 가지 아이디어를 생각해봅시다:
     1. **커플 아이템**: 커플링, 커플 티셔츠, 커플 컵 등 두 사람의 관계를 기념할 수 있는 아이템.
     2. **추억의 앨범**: 함께 보낸 소중한 순간들을 사진으로 모아 앨범을 만들어서 선물하기.
     3. **특별한 데이트**: 그녀가 좋아하는 장소에서의 특별한 데이트 계획이나 소풍, 저녁 식사 예약.
     4. **편지**: 자신의 진솔한 감정을 담은 문자나 손편지를 작성하여 선물과 함께 줍니다.

3. 실행:
   - 먼저, 그녀의 취향을 반영한 선물 아이템을 선정합니다. 항상 일상에서 그녀에게 중요하게 여기는 것이나 좋아하는 것들에 대해 생각해보세요. 다음으로 그 선물을 구매하거나 제작하는 시간을 계획하고, 또한 그날의 데이트 코스를 고민해봅니다. 마지막으로, 짧지만 진솔한 편지를 써서 선물과 함께 드리면 더욱 감동적인 순간을 만들 수 있습니다. 기념일이 다가오니 미리 준비하고 여유롭게 진행해주세요!


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [18]:
def fridge_raid_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    남은 음식 재료를 활용해 최상의 맛을 낼 수 있는 요리의 조리법을 알려주는 냉털마스터입니다.
    웹에서 레시피 검색을 통해 맛있는 요리 하나를 추천해주세요.
    1인분을 만들 것이고, 재료들의 계량도 해야합니다.

    ### Output Format ###
    1. 냉털 메뉴와 재료:

    2. 단계별 조리법:

    3. 추가하면 좋을 재료:

    4. 레시피 출처:
    """

    user_message = f"""
    제공된 재료:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [19]:
print(fridge_raid_coach('제공된 재료: 된장, 돼지고기, 당근'))

1. 냉털 메뉴와 재료:
   - 메뉴: 된장 돼지고기 찌개
   - 재료:
     - 돼지고기: 150g
     - 된장: 2큰술
     - 당근: 50g
     - 물: 300ml
     - 양파 (선택): 1/4개
     - 대파 (선택): 1대
     - 마늘: 1쪽

2. 단계별 조리법:
   1. 돼지고기는 먹기 좋은 크기로 자르고, 당근은 채 썰어 준비합니다. 양파와 대파도 사용한다면 양파는 채 썰고 대파는 어슷하게 썰어줍니다.
   2. 냄비에 돼지고기를 넣고 중약불에서 볶아 고기가 겉면이 살짝 익을 때까지 조리합니다.
   3. 고기가 익으면 양파와 마늘을 넣고 함께 볶아 향을 내줍니다.
   4. 된장을 넣고 중약불에서 잘 저어가며 볶아줍니다.
   5. 볶아진 재료에 물을 붓고 잘 걸쭉해질 때까지 끓입니다. (약 10분 정도)
   6. 국물이 끓으면 준비한 당근과 대파를 넣고 다시 끓입니다.
   7. 재료가 충분히 익고 국물이 우러나면 불을 끄고 맛을 봐서 필요 시 간을 맞춥니다.
   8. 완성된 찌개를 그릇에 담고 뜨거울 때 밥과 함께 제공합니다.

3. 추가하면 좋을 재료:
   - 두부: 1/4 모 (입맛에 따라 추가)
   - 청양고추: 1개 (매운 맛을 원할 때)
   - 버섯: 50g (식감을 추가할 때)

4. 레시피 출처:
   - [네이버 블로그: '된장 찌개 만들기'](https://blog.naver.com) (본 레시피는 블로그의 다양한 된장찌개 레시피를 참고하여 작성하였습니다.)


In [21]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_coach(user_foods)))

1. 냉털 메뉴와 재료:
   - **된장 소고기 볶음** (1인분)
   - 재료:
     - 소고기: 100g
     - 양파: 1/2개 (약 50g)
     - 표고버섯: 2~3개 (약 30g)
     - 명란젓: 1큰술 (약 15g)
     - 된장: 1큰술 (약 15g)
     - 식용유: 1큰술
     - 물: 100ml
     - 후추: 약간

2. 단계별 조리법:
   1. 양파는 얇게 채 썰고, 표고버섯은 가늘게 슬라이스합니다.
   2. 프라이팬에 식용유를 두르고 중불로 예열한 후, 양파를 넣고 투명해질 때까지 볶습니다.
   3. 양파가 투명해지면 소고기를 추가하여 고기가 거의 익을 때까지 볶습니다.
   4. 그 다음 표고버섯을 넣고 섞어가며 약 2~3분 더 볶습니다.
   5. 볶은 재료에 명란젓과 된장을 넣고 잘 섞은 후, 물 100ml를 부어 끓입니다.
   6. 국물이 약간 졸아들 때까지 중약불에서 5분 정도 조리한 후, 후추로 간을 맞추고 불을 끕니다.

3. 추가하면 좋을 재료:
   - 청양고추: 매운 맛을 더해줄 수 있습니다.
   - 참기름: 풍미를 더해줍니다.
   - 대파: 고명으로 추가하면 색다른 맛을 낼 수 있습니다.

4. 레시피 출처:
   - 자신만의 조합, 개인적인 경험을 바탕으로 작성한 레시피입니다.

## 면접질문 생성 JSON 출력


In [22]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [26]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "Python을 사용하여 머신러닝 모델의 배포를 위한 코드를 작성한 경험에 대해 설명해주세요.",
            "answer": "저는 Python을 사용하여 머신러닝 모델을 배포하기 위한 Flask API를 구축한 경험이 있습니다. 모델 학습 후, 해당 모델을 Flask 서버에 업로드하고, RESTful API를 통해 예측 요청을 처리할 수 있도록 구현했습니다. 이를 통해 모델 업데이트 및 배포 절차를 간소화하고, 실시간 예측을 가능하게 만들었습니다."
        },
        {
            "question": "Kubernetes를 사용하여 머신러닝 인프라를 운영한 경험에 대해 말씀해 주세요.",
            "answer": "Kubernetes를 활용하여 머신러닝 모델을 컨테이너화하고 클러스터에 배포한 경험이 있습니다. 노드를 설정하고, Helm을 사용하여 다양한 머신러닝 서비스들을 관리했습니다. 이를 통해 여러 모델의 가용성을 높이더라도 시스템 성능이 유지될 수 있도록 하였으며, 자동화된 스케일링을 사용하여 수요에 맞춘 리소스를 제공했습니다."
        },
        {
            "question": "CI/CD 도구(Jenkins, ArgoCD 등)를 활용한 프로젝트 경험이 있다면 말씀해 주세요.",
            "answer": "지난 프로젝트에서는 Jenkins를 구축하여 머신러닝 모델의 CI/CD 파이프라인을 만들었습니다. 소스 코드 변경 사항이 감지되면 자동으로 테스트와 빌드가 수행되고, 문제가 없으면 바로 Kubernetes 클러스터로 배포되도록 설정했습니다. 이를 통해 배포 시간을 크게 단축시킬 수 있었고, 신뢰성을 높일 수 있었습니다."
        }
    ],
    "soft_skill": [
        {
            "

In [28]:
import json

# 응답데이터가 json형식이 아니라면 오류발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']

print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'Python을 사용하여 머신러닝 모델의 배포를 위한 코드를 작성한 경험에 대해 설명해주세요.', 'answer': '저는 Python을 사용하여 머신러닝 모델을 배포하기 위한 Flask API를 구축한 경험이 있습니다. 모델 학습 후, 해당 모델을 Flask 서버에 업로드하고, RESTful API를 통해 예측 요청을 처리할 수 있도록 구현했습니다. 이를 통해 모델 업데이트 및 배포 절차를 간소화하고, 실시간 예측을 가능하게 만들었습니다.'}, {'question': 'Kubernetes를 사용하여 머신러닝 인프라를 운영한 경험에 대해 말씀해 주세요.', 'answer': 'Kubernetes를 활용하여 머신러닝 모델을 컨테이너화하고 클러스터에 배포한 경험이 있습니다. 노드를 설정하고, Helm을 사용하여 다양한 머신러닝 서비스들을 관리했습니다. 이를 통해 여러 모델의 가용성을 높이더라도 시스템 성능이 유지될 수 있도록 하였으며, 자동화된 스케일링을 사용하여 수요에 맞춘 리소스를 제공했습니다.'}, {'question': 'CI/CD 도구(Jenkins, ArgoCD 등)를 활용한 프로젝트 경험이 있다면 말씀해 주세요.', 'answer': '지난 프로젝트에서는 Jenkins를 구축하여 머신러닝 모델의 CI/CD 파이프라인을 만들었습니다. 소스 코드 변경 사항이 감지되면 자동으로 테스트와 빌드가 수행되고, 문제가 없으면 바로 Kubernetes 클러스터로 배포되도록 설정했습니다. 이를 통해 배포 시간을 크게 단축시킬 수 있었고, 신뢰성을 높일 수 있었습니다.'}]
[{'question': '팀 내 알고리즘 개발자와의 기술 지원을 위해 어떤 커뮤니케이션 전략을 사용하나요?', 'answer': '저는 팀 내에서 정기적인 회의를 통해 알고리즘 개발자와의 소통을 강화합니다. 각자가 직면한 문제와 진행 상황을 공유함으로써, 필요한 기술 지원을 적시에 제공할 수 있습니다. 또한, 문서화된 자료를 통해 쉽게 접근할 수